In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_community.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv


load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
print(KEY)

sk-proj-mBEFjwtjwQYhnO4V3jPLT3BlbkFJboHaJAxXbKkdFwXVPJPL


In [6]:
llm=ChatOpenAI(open_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.3)

c:\Users\JOHN\mcqgen1\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! open_api_key is not default parameter.
                    open_api_key was transferred to model_kwargs.
                    Please confirm that open_api_key is what you intended.


In [7]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000279E4AD4610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000279E4AE2490>, temperature=0.3, model_kwargs={'open_api_key': 'sk-proj-mBEFjwtjwQYhnO4V3jPLT3BlbkFJboHaJAxXbKkdFwXVPJPL'}, openai_api_key='sk-proj-mBEFjwtjwQYhnO4V3jPLT3BlbkFJboHaJAxXbKkdFwXVPJPL', openai_proxy='')

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}

In [10]:
TEMPLATE= """
Text:{text} 
You are an expert MCQ maker. Given the above text,it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and the check all the questions to be conforming the text as well.
Make sure to format your response as RESPONSE_JSON below and use it as a guide. \
Ensure to make the {number} MCQs
### RESPONSE_JSON
{response_json}


"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

c:\Users\JOHN\mcqgen1\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["{subject}", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review",verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [17]:
file_path=r"C:\Users\JOHN\mcqgen\data.txt"

In [18]:
file_path

'C:\\Users\\JOHN\\mcqgen\\data.txt'

In [19]:
with open(file_path, 'r') as file:
    TEXT=file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\JOHN\\mcqgen\\data.txt'

In [ ]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [ ]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [ ]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

TypeError: create() got an unexpected keyword argument 'open_api_key'

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [ ]:
response

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text} \nYou are an expert MCQ maker. Given the above text,it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repeated and the check all the questions to be conforming the text as well.\nMake sure to format your response as RESPONSE_JSON below and use it as a guide. Ensure to make the {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022FB92B2A90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022FB92AD820>, temperature=0.3, model_kwargs={'open_api_key': 'sk-proj-mBEFjwtjwQYhnO4V3jPLT3BlbkFJboHaJAxXbKkdFwXVPJPL'}, openai_api_key='sk-proj-mBEFjwtjwQYhnO4V3jPLT3BlbkFJboHaJAxXbKkdFwXVPJPL', openai_proxy=

In [ ]:
quiz=response.get("quiz")

AttributeError: 'SequentialChain' object has no attribute 'get'

In [ ]:
quiz=json.loads(quiz)

NameError: name 'quiz' is not defined

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

NameError: name 'quiz' is not defined

In [ ]:
quiz_table_data

[]

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("trialquiz.csv", index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'06_08_2024_13_52_39'